In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
df = pd.read_csv('data.csv')

In [84]:
df = df[(df['iso_code']=='FRA') | (df['iso_code']=='GBR') | (df['iso_code']=='ITA')|
        (df['iso_code']=='CHE') | (df['iso_code']=='DEU') | (df['iso_code']=='PRT') ]

In [85]:
df = df[(df['date'] >= '2020-03-15') & (df['date'] <= '2020-11-09')]

In [86]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
17891,FRA,Europe,France,2020-03-15,4499.0,838.0,540.429,91.0,12.0,11.571,...,38605.671,NaN,86.06,4.77,30.1,35.6,NaN,5.98,82.66,0.901
17892,FRA,Europe,France,2020-03-16,5423.0,924.0,613.857,127.0,36.0,15.429,...,38605.671,NaN,86.06,4.77,30.1,35.6,NaN,5.98,82.66,0.901
17893,FRA,Europe,France,2020-03-17,6633.0,1210.0,745.857,148.0,21.0,16.857,...,38605.671,NaN,86.06,4.77,30.1,35.6,NaN,5.98,82.66,0.901
17894,FRA,Europe,France,2020-03-18,7730.0,1097.0,849.429,175.0,27.0,20.286,...,38605.671,NaN,86.06,4.77,30.1,35.6,NaN,5.98,82.66,0.901
17895,FRA,Europe,France,2020-03-19,9134.0,1404.0,979.000,244.0,69.0,28.000,...,38605.671,NaN,86.06,4.77,30.1,35.6,NaN,5.98,82.66,0.901


In [87]:
dfSub = df[['location', 'date','new_cases','population', 'population_density']]

In [88]:
#dfSub.to_csv(r'relevantData.csv', index = False)

In [89]:
A = dfSub[dfSub['location']!='France']

In [90]:
A = A[['location','new_cases','population']]

In [91]:
germ = A[A['location']=='Germany'].reset_index().drop('index',axis=1)
ita = A[A['location']=='Italy'].reset_index().drop('index',axis=1)
port = A[A['location']=='Portugal'].reset_index().drop('index',axis=1)
swi = A[A['location']=='Switzerland'].reset_index().drop('index',axis=1)
unk = A[A['location']=='United Kingdom'].reset_index().drop('index',axis=1)

In [92]:
N = pd.DataFrame(columns=['Italy','United Kingdom'],index = range(0,240))
N['United Kingdom'] = unk['new_cases']
N['Italy']= ita['new_cases']
#N['Portugal'] = port['new_cases']
#N['Switzerland'] = swi['new_cases']
#N['Germany'] = germ['new_cases']

N

,Italy,United Kingdom
0,3497.0,478.0
1,2823.0,361.0
2,4000.0,442.0
3,3526.0,611.0
4,4207.0,769.0
...,...,...
235,30548.0,25177.0
236,34502.0,24138.0
237,37802.0,23287.0
238,39809.0,24957.0


In [93]:
A = N.to_numpy()
A

array([[ 3497.,   478.],
       [ 2823.,   361.],
       [ 4000.,   442.],
       [ 3526.,   611.],
       [ 4207.,   769.],
       [ 5322.,   999.],
       [ 5986.,  1055.],
       [ 6557.,  1255.],
       [ 5560.,  1198.],
       [ 4789.,  1378.],
       [ 5249.,  2338.],
       [ 5210.,  2375.],
       [ 6153.,  2692.],
       [ 5959.,  3087.],
       [ 5974.,  3197.],
       [ 5217.,  2822.],
       [ 4050.,  2858.],
       [ 4053.,  4273.],
       [ 4782.,  4514.],
       [ 4668.,  4913.],
       [ 4585.,  4868.],
       [ 4805.,  4911.],
       [ 4316.,  4020.],
       [ 3599.,  3592.],
       [ 3039.,  5282.],
       [ 3836.,  5450.],
       [ 4204.,  5131.],
       [ 3951.,  4858.],
       [ 4694.,  4313.],
       [ 4092.,  3579.],
       [ 3153.,  3489.],
       [ 2972.,  4178.],
       [ 2667.,  4326.],
       [ 3786.,  5065.],
       [ 3493.,  5292.],
       [ 3491.,  4956.],
       [ 3047.,  4721.],
       [ 2256.,  3853.],
       [ 2729.,  4854.],
       [ 3370.,  4760.],


In [104]:
b = dfSub[dfSub['location']=='France']

In [105]:
b = b[['new_cases']]

In [106]:
b=b.to_numpy()
for i in range(0,240):
    if b[i] == 0:
        b[i]= b[i-1]
    elif b[i]<0:
        b[i]= -b[i]
b

array([[8.3800e+02],
       [9.2400e+02],
       [1.2100e+03],
       [1.0970e+03],
       [1.4040e+03],
       [1.8610e+03],
       [1.6170e+03],
       [1.8470e+03],
       [1.5590e+03],
       [3.8380e+03],
       [2.4460e+03],
       [2.9310e+03],
       [3.9220e+03],
       [3.8090e+03],
       [4.6110e+03],
       [2.5990e+03],
       [4.3760e+03],
       [7.5780e+03],
       [4.8610e+03],
       [2.1160e+03],
       [5.2330e+03],
       [4.2670e+03],
       [1.8730e+03],
       [3.9120e+03],
       [3.7770e+03],
       [3.8810e+03],
       [4.2860e+03],
       [4.3420e+03],
       [3.1140e+03],
       [1.6130e+03],
       [2.6730e+03],
       [5.4970e+03],
       [2.6330e+03],
       [2.6410e+03],
       [4.0500e+02],
       [2.5690e+03],
       [7.8500e+02],
       [2.0510e+03],
       [2.6670e+03],
       [1.8270e+03],
       [1.6530e+03],
       [1.7730e+03],
       [1.5370e+03],
       [4.6100e+02],
       [1.1950e+03],
       [1.0650e+03],
       [1.6070e+03],
       [1.139

In [124]:
A_train = A[:190]
b_train = b[:190]
A_test = A[190:] ## ten days in november 2020
b_test = b[190:]

In [125]:
 x = (np.linalg.inv((A_train.transpose()@A_train)))@A_train.transpose()@b_train
    ##minimization (by def), using train only

In [126]:
x 

array([[0.26143705],
       [0.77982009]])

In [127]:
predictions = A_test@x


In [128]:
predictions

array([[ 3455.41914039],
       [ 3759.19418366],
       [ 4205.05271477],
       [ 5246.48529682],
       [ 5640.25306851],
       [ 5859.57113896],
       [ 5199.51902817],
       [ 4900.43379956],
       [ 3544.17940841],
       [ 6000.84174628],
       [ 6026.61976389],
       [ 6057.81772575],
       [ 6087.11759541],
       [10780.32995146],
       [18579.4338759 ],
       [10410.33785497],
       [12040.01077744],
       [12005.11619062],
       [14843.53080112],
       [12215.8656003 ],
       [13322.43738294],
       [11464.24477974],
       [12103.22407273],
       [14979.81556245],
       [17298.02796299],
       [17100.85607192],
       [14809.21058041],
       [15466.67048706],
       [16302.24566068],
       [17103.99493957],
       [19475.64474825],
       [24784.64053523],
       [20765.20401211],
       [21015.17576028],
       [23079.84362518],
       [20994.18998476],
       [20736.70163783],
       [23594.92210066],
       [25794.34527915],
       [25000.64503307],


In [129]:
b_test

array([[11569.],
       [ 4298.],
       [10008.],
       [13072.],
       [16096.],
       [15797.],
       [14412.],
       [11123.],
       [ 4070.],
       [ 8051.],
       [12845.],
       [13970.],
       [12148.],
       [16972.],
       [12545.],
       [ 5104.],
       [10489.],
       [18746.],
       [18129.],
       [20339.],
       [26896.],
       [16101.],
       [ 8505.],
       [12993.],
       [22591.],
       [30621.],
       [25086.],
       [32427.],
       [29837.],
       [13243.],
       [20468.],
       [26676.],
       [41622.],
       [42032.],
       [45422.],
       [52010.],
       [26771.],
       [33417.],
       [36437.],
       [47637.],
       [49215.],
       [32641.],
       [49290.],
       [52518.],
       [36330.],
       [40558.],
       [58046.],
       [60486.],
       [86852.],
       [38619.]])

In [133]:
predictions - b_test

array([[ -8113.58085961],
       [  -538.80581634],
       [ -5802.94728523],
       [ -7825.51470318],
       [-10455.74693149],
       [ -9937.42886104],
       [ -9212.48097183],
       [ -6222.56620044],
       [  -525.82059159],
       [ -2050.15825372],
       [ -6818.38023611],
       [ -7912.18227425],
       [ -6060.88240459],
       [ -6191.67004854],
       [  6034.4338759 ],
       [  5306.33785497],
       [  1551.01077744],
       [ -6740.88380938],
       [ -3285.46919888],
       [ -8123.1343997 ],
       [-13573.56261706],
       [ -4636.75522026],
       [  3598.22407273],
       [  1986.81556245],
       [ -5292.97203701],
       [-13520.14392808],
       [-10276.78941959],
       [-16960.32951294],
       [-13534.75433932],
       [  3860.99493957],
       [  -992.35525175],
       [ -1891.35946477],
       [-20856.79598789],
       [-21016.82423972],
       [-22342.15637482],
       [-31015.81001524],
       [ -6034.29836217],
       [ -9822.07789934],
       [-106

In [132]:
predictions - b_test

array([[ -8113.58085961],
       [  -538.80581634],
       [ -5802.94728523],
       [ -7825.51470318],
       [-10455.74693149],
       [ -9937.42886104],
       [ -9212.48097183],
       [ -6222.56620044],
       [  -525.82059159],
       [ -2050.15825372],
       [ -6818.38023611],
       [ -7912.18227425],
       [ -6060.88240459],
       [ -6191.67004854],
       [  6034.4338759 ],
       [  5306.33785497],
       [  1551.01077744],
       [ -6740.88380938],
       [ -3285.46919888],
       [ -8123.1343997 ],
       [-13573.56261706],
       [ -4636.75522026],
       [  3598.22407273],
       [  1986.81556245],
       [ -5292.97203701],
       [-13520.14392808],
       [-10276.78941959],
       [-16960.32951294],
       [-13534.75433932],
       [  3860.99493957],
       [  -992.35525175],
       [ -1891.35946477],
       [-20856.79598789],
       [-21016.82423972],
       [-22342.15637482],
       [-31015.81001524],
       [ -6034.29836217],
       [ -9822.07789934],
       [-106